### Porting to Google Colab
The following cell enables this notebook to run from Google Colab as well as from your local machine IDE.<br>
You can change `root_directory` and/or `this_notebook_google_path` to point to the directory in your Google account, which contains this notebook, together with the `imgs` sub-directory and the rest of the files.<br>

In [1]:
import sys
import os
try:
    from google.colab import drive as google_drive # type: ignore
except:
    # no Google Colab --> fall back to local machine
    google_drive = None

if google_drive is not None:
    google_drive_directory = os.path.join('/','content','gdrive')
    google_drive.mount(google_drive_directory)
    all_projects_path = os.path.join(google_drive_directory, 'Othercomputers','My Laptop', 'projects')
else:
    all_projects_path = os.path.join('d:\\', 'projects')

project_path = os.path.join(all_projects_path,'RUNI','Thesis')
assert os.path.exists(project_path), f'Project path {project_path} not found!'
# enable import python files from this notebook's path
sys.path.append(project_path)
# enable reading images and data files from this notebook's path
os.chdir(project_path)

datasets_path = os.path.join(project_path, 'datasets')
assert os.path.exists(datasets_path), f'Datasets path {datasets_path} not found!'

output_path = os.path.join(project_path, 'output')
os.makedirs(output_path, exist_ok=True)
assert os.path.exists(output_path), f'Output path {output_path} not found!'

print(f'Current working directory: {os.getcwd()}')
print(f'Datasets path: {datasets_path}')
print(f'Output path: {output_path}')

Current working directory: d:\projects\RUNI\Thesis
Datasets path: d:\projects\RUNI\Thesis\datasets
Output path: d:\projects\RUNI\Thesis\output


In [2]:
from python.hpc import HybridArray

Numba version: 0.60.0
numba.njit is available.
CUDA is available and will be used for GPU operations.
Printing CUDA active device attributes:
    Name:                               NVIDIA GeForce GTX 1650
    Free Memory:                        3367680 [KB]
    Total Memory:                       4193984 [KB]
    Compute capability:                 7.5
    Clock rate:                         1560.00 [MHz]
    Memory clock rate:                  4001.00 [MHz]
    Memory bus width:                   128 bits
    Memory band width (theoretical)     128.03 [GByte/Sec]
    Number of multiprocessors:          16
    Minimal grid size:                  128
    Maximum grid size:                  (2147483647, 65535, 65535)
    Maximum block dimensions:           (1024, 1024, 64)
    Maximum threads per block:          1024
    Warp size:                          32
    Maximum shared memory per block:    49152 [bytes]
    Maximum registers per block:        65536
    Total constant memory:   

In [3]:
import numpy as np
from python.rare_weak_model.rare_weak_model import rare_weak_model, rare_weak_null_hypothesis
from python.adaptive_methods.adaptive_methods import apply_transform_method, discover_by_method
from python.metrics.metrics import detect_signal_auc, create_noise_4_auc, create_signal_4_auc
from python.array_math_utils.array_math_utils import array_transpose_inplace, sort_rows_inplace

In [4]:
shape=(20,100)
num_steps = 3
epsilon = 0.1
mu = 1.0


In [5]:

def simulation(shape: tuple, gpu: bool, njit: bool, transform_method: str, discovery_method: str,\
               num_steps: int|None=None, epsilon: float = 0.01, mu: float = 1.0) -> tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    with (
        HybridArray() as signal,
        HybridArray() as counts,
        HybridArray() as num_discoveries,
        HybridArray() as noise,
        HybridArray() as auc_row
    ):
        create_noise_4_auc(noise=noise, shape=shape, transform_method=transform_method,\
                           use_gpu=gpu, use_njit=njit, num_steps=num_steps)
        create_signal_4_auc(signal=signal, counts=counts, shape=shape,\
                            transform_method=transform_method, ind_model=1,\
                            epsilon=epsilon, mu=mu, use_gpu=gpu, use_njit=njit, num_steps=num_steps)
        discover_by_method(transformed_p_values_input=signal,\
                           num_discoveries_output=num_discoveries,\
                           discover_method=discovery_method,\
                           use_njit=njit)
        auc_row.realloc(like=signal, shape=(1,shape[1]))
        detect_signal_auc(\
            noise_input=noise,\
            signal_input_work=signal,\
            auc_out_row=auc_row,\
            use_njit=njit)
        ret = (signal.numpy(), counts.numpy(), num_discoveries.numpy(), auc_row.numpy())
    return ret

def simulation3(shape: tuple, transform_method: str, discovery_method: str, num_steps: int=3,\
                epsilon: float = 0.01, mu: float = 1.0) -> None:
    print(f'Running on {transform_method=} {discovery_method=} {shape=} {epsilon=} {mu=} {num_steps=}')
    for gpu,njit in [(False,False), (False,True), (True,False)]:
        p_values, counts, num_discoveries, auc =\
            simulation(shape=shape, gpu=gpu, njit=njit,\
                       transform_method=transform_method,\
                       discovery_method=discovery_method,\
                       num_steps=num_steps, epsilon=epsilon, mu=mu)
        print(f'{gpu=} {njit=} --> p_values.mean={p_values.mean():.2e} counts.mean={counts.mean():.2f} num_discoveries.mean={num_discoveries.mean():.2f} auc.mean()={auc.mean():.2f}')




In [6]:
simulation3(shape=shape, transform_method='higher_criticism_stable',\
            discovery_method='argmin', num_steps=num_steps, epsilon=epsilon, mu=mu)

Running on transform_method='higher_criticism_stable' discovery_method='argmin' shape=(20, 100) epsilon=0.1 mu=1.0 num_steps=3
gpu=False njit=False --> p_values.mean=-3.05e+00 counts.mean=5.14 num_discoveries.mean=33.82 auc.mean()=0.76
gpu=False njit=True --> p_values.mean=-3.05e+00 counts.mean=5.14 num_discoveries.mean=33.82 auc.mean()=0.76


d:\venv\thesis\Lib\site-packages\numba\cuda\dispatcher.py:536: NumbaPerformanceWarning: Grid size 2 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
d:\venv\thesis\Lib\site-packages\numba\cuda\dispatcher.py:536: NumbaPerformanceWarning: Grid size 2 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
d:\venv\thesis\Lib\site-packages\numba\cuda\dispatcher.py:536: NumbaPerformanceWarning: Grid size 2 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
d:\venv\thesis\Lib\site-packages\numba\cuda\dispatcher.py:536: NumbaPerformanceWarning: Grid size 2 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
d:\venv\thesis\Lib\site-packages\numba\cuda\dispatcher.py:536: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanc

gpu=True njit=False --> p_values.mean=3.69e-01 counts.mean=5.14 num_discoveries.mean=33.82 auc.mean()=0.76


d:\venv\thesis\Lib\site-packages\numba\cuda\dispatcher.py:536: NumbaPerformanceWarning: Grid size 2 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


In [7]:
simulation3(shape=shape, transform_method='higher_criticism_unstable',\
            discovery_method='argmin', num_steps=num_steps, epsilon=epsilon, mu=mu)

Running on transform_method='higher_criticism_unstable' discovery_method='argmin' shape=(20, 100) epsilon=0.1 mu=1.0 num_steps=3
gpu=False njit=False --> p_values.mean=-3.97e+00 counts.mean=5.14 num_discoveries.mean=13.05 auc.mean()=0.76
gpu=False njit=True --> p_values.mean=-3.97e+00 counts.mean=5.14 num_discoveries.mean=13.05 auc.mean()=0.76
gpu=True njit=False --> p_values.mean=3.69e-01 counts.mean=5.14 num_discoveries.mean=13.05 auc.mean()=0.76


d:\venv\thesis\Lib\site-packages\numba\cuda\dispatcher.py:536: NumbaPerformanceWarning: Grid size 2 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
d:\venv\thesis\Lib\site-packages\numba\cuda\dispatcher.py:536: NumbaPerformanceWarning: Grid size 2 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


In [8]:
simulation3(shape=shape, transform_method='berk_jones',\
            discovery_method='argmin', num_steps=num_steps, epsilon=epsilon, mu=mu)

Running on transform_method='berk_jones' discovery_method='argmin' shape=(20, 100) epsilon=0.1 mu=1.0 num_steps=3
gpu=False njit=False --> p_values.mean=2.97e-01 counts.mean=5.14 num_discoveries.mean=31.42 auc.mean()=0.76
gpu=False njit=True --> p_values.mean=2.97e-01 counts.mean=5.14 num_discoveries.mean=31.42 auc.mean()=0.76


d:\venv\thesis\Lib\site-packages\numba\cuda\dispatcher.py:536: NumbaPerformanceWarning: Grid size 14 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


gpu=True njit=False --> p_values.mean=3.69e-01 counts.mean=5.14 num_discoveries.mean=31.42 auc.mean()=0.76


d:\venv\thesis\Lib\site-packages\numba\cuda\dispatcher.py:536: NumbaPerformanceWarning: Grid size 2 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
